I used ChatGPT. I asked "Make a python script that simulates a customer service flow."


In [ ]:
#!/usr/bin/env python3
"""
customer_service_sim.py

A lightweight CLI simulation of a customer service flow:
- Greeting & customer capture
- Intent detection from free text (billing / technical / order / other)
- Branch-specific Q&A
- Troubleshoot or resolve using a tiny knowledge base
- Escalate when needed
- CSAT survey
- Save a transcript & mock ticket

Run:  python customer_service_sim.py
"""

import json
import random
import textwrap
from datetime import datetime
from pathlib import Path

# -----------------------------
# Tiny “knowledge base” examples
# -----------------------------
KB = {
    "billing:refund_policy": (
        "Our standard refund window is 30 days from the charge date. "
        "If you're within that window and the service wasn't used, "
        "we can process a full refund."
    ),
    "billing:update_payment": (
        "To update your payment method, go to Account → Billing → Payment Methods "
        "and add a new card. You can set it as default to use it next cycle."
    ),
    "technical:app_crash": (
        "Please try: 1) Force-quit the app, 2) Clear cache/storage, 3) Reinstall. "
        "If the crash persists, capture logs (Settings → Advanced → Export Logs) "
        "and share the file with support."
    ),
    "technical:login_issue": (
        "If you see a login error, reset your password from the login screen. "
        "If 2FA is enabled, ensure your device time is automatic. "
        "VPNs or ad-blockers can also interfere—try disabling temporarily."
    ),
    "order:track_package": (
        "You can track the package from Account → Orders → 'Track'. "
        "Most carriers update statuses every 4–12 hours."
    ),
    "order:change_address": (
        "If the order hasn't shipped, you can change the address from Account → Orders. "
        "After shipment, we can request a carrier intercept (not guaranteed)."
    ),
}

# -----------------------------
# Simple keyword-based intents
# -----------------------------
INTENT_KEYWORDS = {
    "billing": {"charge", "refund", "invoice", "payment", "bill", "subscription", "price"},
    "technical": {"error", "bug", "issue", "crash", "login", "slow", "not working", "can't", "cannot"},
    "order": {"order", "shipping", "delivery", "package", "address", "tracking"},
}

# -----------------------------
# Utilities
# -----------------------------
def wrap(msg: str) -> str:
    return textwrap.fill(msg, width=88)

def ask(prompt: str) -> str:
    print()
    print(wrap(prompt))
    return input("> ").strip()

def choose(prompt: str, options: list[str]) -> str:
    print()
    print(wrap(prompt))
    for i, opt in enumerate(options, 1):
        print(f"  {i}. {opt}")
    while True:
        sel = input("> ").strip()
        if sel.isdigit() and 1 <= int(sel) <= len(options):
            return options[int(sel) - 1]
        print("Please enter a valid number from the list.")

def detect_intent(text: str) -> str:
    t = text.lower()
    best_intent, best_score = "other", 0
    for intent, keywords in INTENT_KEYWORDS.items():
        score = sum(1 for kw in keywords if kw in t)
        if score > best_score:
            best_intent, best_score = intent, score
    return best_intent

def new_ticket_id() -> str:
    return f"TCK-{datetime.utcnow().strftime('%Y%m%d%H%M%S')}-{random.randint(100,999)}"

# -----------------------------
# Flow
# -----------------------------
class CustomerServiceSim:
    def __init__(self):
        self.transcript: list[dict] = []
        self.customer = {"name": None, "email": None}
        self.intent = "other"
        self.ticket_id = None
        self.status = "new"  # new → in_progress → resolved/escalated → closed

    def say(self, role: str, text: str):
        print()
        print(wrap(text))
        self.transcript.append({
            "ts": datetime.utcnow().isoformat(),
            "role": role,
            "text": text
        })

    def capture_customer(self):
        self.say("agent", "Hi! Thanks for contacting support. I’m Ava. What’s your name?")
        self.customer["name"] = input("> ").strip() or "Customer"
        self.transcript.append({"ts": datetime.utcnow().isoformat(), "role": "user", "text": self.customer["name"]})

        self.customer["email"] = ask(f"Nice to meet you, {self.customer['name']}! What’s the best email to reach you?")
        self.transcript.append({"ts": datetime.utcnow().isoformat(), "role": "user", "text": self.customer["email"]})

    def capture_issue(self):
        issue = ask("Briefly describe what you need help with today:")
        self.transcript.append({"ts": datetime.utcnow().isoformat(), "role": "user", "text": issue})
        self.intent = detect_intent(issue)
        self.say("agent", f"Got it—sounds like {self.intent} support. I’ll take a look.")

    # --- Branches ---
    def handle_billing(self):
        choice = choose(
            "Billing topics I can help with right now:",
            ["Refund policy", "Update payment method", "Something else"]
        )
        if choice == "Refund policy":
            self.say("agent", KB["billing:refund_policy"])
            return "resolved"
        elif choice == "Update payment method":
            self.say("agent", KB["billing:update_payment"])
            return "resolved"
        else:
            notes = ask("Tell me more about the billing issue:")
            self.transcript.append({"ts": datetime.utcnow().isoformat(), "role": "user", "text": notes})
            self.say("agent", "Thanks—I may need a specialist to review this.")
            return "escalate"

    def handle_technical(self):
        choice = choose(
            "Technical topics I can help with:",
            ["App is crashing", "Login trouble", "Something else"]
        )
        if choice == "App is crashing":
            self.say("agent", KB["technical:app_crash"])
            worked = choose("Did that help?", ["Yes, resolved", "No, still broken"])
            return "resolved" if worked == "Yes, resolved" else "escalate"
        elif choice == "Login trouble":
            self.say("agent", KB["technical:login_issue"])
            worked = choose("Did that help?", ["Yes, resolved", "No, still blocked"])
            return "resolved" if worked == "Yes, resolved" else "escalate"
        else:
            details = ask("Describe the issue and any error messages:")
            self.transcript.append({"ts": datetime.utcnow().isoformat(), "role": "user", "text": details})
            self.say("agent", "Thanks—I’ll route this to an engineer.")
            return "escalate"

    def handle_order(self):
        choice = choose(
            "Order topics I can help with:",
            ["Track a package", "Change shipping address", "Something else"]
        )
        if choice == "Track a package":
            order_id = ask("What’s your order number?")
            self.transcript.append({"ts": datetime.utcnow().isoformat(), "role": "user", "text": order_id})
            self.say("agent", KB["order:track_package"])
            return "resolved"
        elif choice == "Change shipping address":
            stage = choose("Has the order shipped yet?", ["Not shipped", "Already shipped / unknown"])
            if stage == "Not shipped":
                self.say("agent", KB["order:change_address"])
                return "resolved"
            else:
                self.say("agent", "I’ll request a carrier intercept, but it’s not guaranteed.")
                return "escalate"
        else:
            notes = ask("Tell me what you need help with on the order:")
            self.transcript.append({"ts": datetime.utcnow().isoformat(), "role": "user", "text": notes})
            self.say("agent", "Thanks—this may require a fulfillment specialist.")
            return "escalate"

    def handle_other(self):
        notes = ask("Thanks! Could you share a few details?")
        self.transcript.append({"ts": datetime.utcnow().isoformat(), "role": "user", "text": notes})
        self.say("agent", "I’ll find the right person to help.")
        return "escalate"

    def branch_flow(self):
        self.status = "in_progress"
        if self.intent == "billing":
            return self.handle_billing()
        elif self.intent == "technical":
            return self.handle_technical()
        elif self.intent == "order":
            return self.handle_order()
        return self.handle_other()

    def escalate(self):
        self.ticket_id = new_ticket_id()
        self.say(
            "agent",
            f"I’m escalating this to a specialist. Your ticket number is {self.ticket_id}. "
            "You’ll hear from us by email soon."
        )
        self.status = "escalated"

    def csat(self):
        score = choose("Before you go, how satisfied are you with today’s support?", ["1", "2", "3", "4", "5"])
        comment = ask("Any quick feedback for us? (optional)")
        self.transcript.append({"ts": datetime.utcnow().isoformat(), "role": "user", "text": f"CSAT {score}; {comment}"})
        self.say("agent", "Thank you! Your feedback helps us improve.")

    def save_transcript(self, filename: str | None = None):
        record = {
            "ticket_id": self.ticket_id or new_ticket_id(),
            "customer": self.customer,
            "intent": self.intent,
            "status": self.status,
            "created_utc": datetime.utcnow().isoformat(),
            "transcript": self.transcript,
        }
        if not filename:
            filename = f"transcript_{record['ticket_id']}.json"
        Path(filename).write_text(json.dumps(record, indent=2))
        self.say("system", f"Transcript saved to {filename}")

    def run(self):
        self.capture_customer()
        self.capture_issue()
        outcome = self.branch_flow()
        if outcome == "resolved":
            self.status = "resolved"
            self.say("agent", "Happy to help! I’m marking this as resolved. 🎉")
        else:
            self.escalate()
        self.csat()
        self.status = "closed" if self.status == "resolved" else self.status
        self.save_transcript()


# -----------------------------
# Main
# -----------------------------
def main():
    sim = CustomerServiceSim()
    sim.run()

if __name__ == "__main__":
    main()


/tmp/ipython-input-604937401.py:114: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "ts": datetime.utcnow().isoformat(),



Hi! Thanks for contacting support. I’m Ava. What’s your name?
> Jt


/tmp/ipython-input-604937401.py:122: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  self.transcript.append({"ts": datetime.utcnow().isoformat(), "role": "user", "text": self.customer["name"]})



Nice to meet you, Jt! What’s the best email to reach you?
> jaretlee@gmail.com


/tmp/ipython-input-604937401.py:125: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  self.transcript.append({"ts": datetime.utcnow().isoformat(), "role": "user", "text": self.customer["email"]})



Briefly describe what you need help with today:
> Refund


/tmp/ipython-input-604937401.py:129: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  self.transcript.append({"ts": datetime.utcnow().isoformat(), "role": "user", "text": issue})



Got it—sounds like billing support. I’ll take a look.

Billing topics I can help with right now:
  1. Refund policy
  2. Update payment method
  3. Something else
> 1

Our standard refund window is 30 days from the charge date. If you're within that window
and the service wasn't used, we can process a full refund.

Happy to help! I’m marking this as resolved. 🎉

Before you go, how satisfied are you with today’s support?
  1. 1
  2. 2
  3. 3
  4. 4
  5. 5
> 4

Any quick feedback for us? (optional)
> no

Thank you! Your feedback helps us improve.

Transcript saved to transcript_TCK-20251007230240-672.json


/tmp/ipython-input-604937401.py:222: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  self.transcript.append({"ts": datetime.utcnow().isoformat(), "role": "user", "text": f"CSAT {score}; {comment}"})
/tmp/ipython-input-604937401.py:97: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return f"TCK-{datetime.utcnow().strftime('%Y%m%d%H%M%S')}-{random.randint(100,999)}"
/tmp/ipython-input-604937401.py:231: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "created_utc": datetime.utcnow().isoformat(),
